# Recommender System
    1) Popularty bases recomandation (generaly based on formula)
    2) Content bases Recommandation 
    3) Collaborative based Recommendation
    4) Hibread based Recomandation

In [1]:
from kaggle_handler import handler
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import pickle

In [4]:
Assets = handler("arashnic/book-recommendation-dataset")

Directory 'Assets' already exists.
Datasets already exist in Assets folder
['DeepRec.png', 'Users.csv', 'recsys_taxonomy2.png', 'Books.csv', 'Ratings.csv', 'classicRec.png']
 Change Add_more parameter to download more datasets


In [5]:
Assets

['DeepRec.png',
 'Users.csv',
 'recsys_taxonomy2.png',
 'Books.csv',
 'Ratings.csv',
 'classicRec.png']

In [6]:
book = pd.read_csv('Assets/Books.csv')
users = pd.read_csv('Assets/Users.csv')
ratings = pd.read_csv('Assets/Ratings.csv')

/tmp/ipykernel_106250/3435027196.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('Assets/Books.csv')


In [7]:
book.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [8]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [9]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [10]:
print(book.shape)
print(users.shape)
print(ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [11]:
book.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [12]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [13]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [14]:
book.duplicated().sum()

np.int64(0)

In [15]:
users.duplicated().sum()

np.int64(0)

In [16]:
ratings.duplicated().sum()

np.int64(0)

## Popularity Based Recommender System
    - Top 50 books
    - Have more then 250 votes

In [17]:
ratings_with_name = ratings.merge(book, on='ISBN')
ratings_with_name.shape

(1031136, 10)

In [18]:
ratings_with_name.head(2)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...


In [19]:
num_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].count().reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df.head(2)

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1


In [20]:
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_ratings'},inplace=True)
avg_rating_df.head(2)

,Book-Title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00


In [21]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df.head(2)

,Book-Title,num_ratings,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.25
1,Always Have Popsicles,1,0.00


In [22]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings', ascending=False).head(50)

In [23]:
popular_df = popular_df.merge(book, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_ratings']]

In [24]:
popular_df.head(5)

,Book-Title,Book-Author,Image-URL-M,num_ratings,avg_ratings
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,556,5.183453


In [25]:
pickle.dump(popular_df,open('top50books.pkl', 'wb'))

## Collaborative Filtering Recommender
    - Recommendation system bases on ever user rating
    - users > 200 && books >= 50

In [26]:
ratings_with_name.head(5)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...


In [27]:
x = ratings_with_name.groupby('User-ID')['Book-Rating'].count() > 200
valid_user = x[x].index

In [28]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(valid_user)]

In [29]:
y = filtered_rating.groupby('Book-Title')['Book-Rating'].count() >= 50
femous_books = y[y].index

In [30]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(femous_books)]

In [31]:
final_ratings.duplicated().sum()

np.int64(0)

In [32]:
final_ratings.shape

(58586, 10)

In [33]:
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')

In [34]:
pt.fillna(0, inplace=True)

In [35]:
pt.shape

(706, 810)

In [36]:
similarity_score = cosine_similarity(pt)

In [37]:
similarity_score.shape

(706, 706)

In [38]:
np.where(pt.index=='4 Blondes')[0][0]

np.int64(3)

In [39]:
def recommend(book_name):
    index = np.where(pt.index==book_name)[0][0]
    distances = similarity_score[index]
    similar_items = sorted(list(enumerate(distances)),key=lambda x:x[1], reverse=True)[1:6]

    for i in similar_items:
        print(pt.index[i[0]])
    # return suggestion

In [40]:
recommend('The Notebook')

A Walk to Remember
The Rescue
One Door Away from Heaven
Toxin
The Five People You Meet in Heaven


In [41]:
book_poster = final_ratings.drop_duplicates(subset=['Book-Title']).reset_index()[['Book-Title', 'Image-URL-M']]

In [42]:
pickle.dump(pt, open('user_matrix.pkl', 'wb'))
pickle.dump(similarity_score, open('similarity.pkl', 'wb'))

In [43]:
pickle.dump(book_poster, open('book_poster.pkl', 'wb'))